In [12]:
import pandas as pd
import re
import msoffcrypto
import io
import openpyxl
import os
import shutil
import requests
import json
import datetime
import itertools
import pickle
import operator
import time
from natasha import (
    Segmenter,
    MorphVocab,
    LOC,
    AddrExtractor    
)
from pandas.tseries.offsets import MonthEnd
from meteostat import Point, Daily, Monthly


from xlrd import XLRDError
from zipfile import BadZipFile
from pathlib import Path
import dateparser
from collections import Counter, defaultdict


In [13]:
def save_dfs(dfs: dict[str, pd.DataFrame], path):
    """
    Сохраняет pandas datafram-ы dfs
    в frt файлы в папке path
    """
    Path(path).mkdir(parents=True, exist_ok=True)
    for path_to_file, df in dfs.items():
        df.to_feather(path=f"{path}/{os.path.basename(path_to_file)}_parsed")

def read_files(dir, regex = r".*"):
    """
    Читает excel файлы в папке и подпапках dir,
    имена которых удовлетворяют regex
    """
    skipped_files = []
    errors = set()

    def handler(path, exc):
        if type(exc) == XLRDError and str(exc) == "Excel xlsx file; not supported":
            return pd.read_excel(path, engine='openpyxl')
        skipped_files.append((str(path), exc.__repr__()))
        errors.add(exc.__repr__())


    def wrapper(dir):
        files = dict()
        regex_engine = [
            (r".*xlsx",'openpyxl'),
            (r".*xls", 'xlrd'),
        ]
        for file_or_dir in dir.iterdir():
            if file_or_dir.is_dir():
                files.update(wrapper(file_or_dir))
            for regex_excel, engine in regex_engine:
                if re.fullmatch(regex_excel, str(file_or_dir)) and re.fullmatch(regex, str(file_or_dir)):
                    try:
                        files[str(file_or_dir)] = pd.read_excel(file_or_dir, engine=engine)
                    except Exception as e:
                        result = handler(file_or_dir, e)
                        if result is not None:
                            files[str(file_or_dir)] = result
        return files
    result = wrapper(dir)
    print(f"Unparsed files: ", *skipped_files, sep='\n')
    print(f"Errors: ", *errors)
    return result


In [14]:
def parse_date(series):
    """
    Парсит нормальные даты и даты типа 'Июнь 2020'
    """
    for i, elem in enumerate(series):
        series.loc[i] = dateparser.parse(elem)
    return series

def extract_year(name):
    """
    Извлекает год из имени файла - 1
    """
    start, _ = re.search(r"01.01.20.*xls.*", name).span()
    return int(name[start + 6: start + 10]) - 1

def natasha_process(text):
     if not text:
          return None
     segmenter = Segmenter()
     morph_vocab = MorphVocab()
     addr_extractor = AddrExtractor(morph_vocab)

     matches = addr_extractor(text)
     facts = [i.fact.as_json for i in matches]
     lst = []
     for fact in facts:
          lst.extend(list(fact.values())[::-1])
     return " ".join(lst)

def extract_string_for_search(s):
    """
    Извлекает подстроку, которая начинается
    с 'г.' или 'город'
    """
    city = re.search("(г\.|город ).*", s)
    
    if city is not None:
        start, end = city.span()
        return s[start:end]
    region = re.search("(адресу:).*", s)

    if region is not None:
        start, end = region.span()
        return s[start + 7:end]
    natasha = natasha_process(s)
    if natasha:
        return natasha
    return s

def add_string_col_to_df(df, raw_string_column="raw_string", result_column="clean_address"):
    """
    Применяет extract_string_for_search к столбцу df[raw_string_column]
    и сохраняет результат в df[result_column]
    """
    df[result_column] = df[raw_string_column].apply(extract_string_for_search)
    return df


In [15]:
def get_next_month(date):
    next_month_date = date + datetime.timedelta(days=31)
    return datetime.date(year=next_month_date.year, month=next_month_date.month, day=1)

def aggregate_by_month(df):
    new_dfs = []
    df["count"] = df['end_date'].dt.to_period("M").astype(int) - df["start_date"].dt.to_period("M").astype(int)
    for _, row in df.iterrows():

        count = row["count"] if row["count"] > 0 else 1
        avg = row["amount_money"] / count
        columns = [col for col in df.columns if col not in ["start_date", "end_date", "count"]]
        columns.append("month")
        new_row = {col: row.get(col) for col in columns}
        new_df = pd.DataFrame(columns=columns, index=range(count))

        start_date = datetime.date(year=row["start_date"].year, month=row["start_date"].month, day=1)
        for i in range(count):
            new_row["amount_money"] = avg
            new_row["month"] = start_date
            new_df.loc[i] = new_row
            start_date = get_next_month(start_date) 
        new_dfs.append(new_df)
    return pd.concat(new_dfs, axis=0, join="outer", ignore_index=True)
        

In [16]:
def process_kr(df):
    """
    Обрабатывает kr df за 2020, 2021, 2022, 2023
    """
    df = df.drop(df.columns[-1], axis = 1)
    columns = df.columns = [
        "Номер проекта/ работы",
        "Наименование проекта/работы",
        "Наименование подразделения Банка России",
        "Общая стоимость",
        "Начало",
        "Окончание",
        "Всего (утверждено)",
        "В том числе по статьям (утверждено)",
        "Всего (на отчетную дату)",
        "В том числе по статьям (на отчетную дату)",
        "% от годового объема затрат (расходов) по актуальной версии плана (на отчетную дату)",
        "Всего (ожидаемое исполнение)",
        "В том числе по статьям (ожидаемое исполнение)",
        "% от годового объема затрат (расходов) по актуальной версии плана (ожидаемое исполнение)",
        "Код состояния проекта/работы",
        "Документ",
        "Дата",
        ]
    columns_cast_to_str = [
        "Начало",
        "Окончание",
    ]
    for column in columns_cast_to_str:
        df[column] = df[column].astype("string")
    df = df[df[columns[0]].isna() == False]
    df = df.reset_index(drop=True)
    df = df.drop([0, 1])
    df = df.reset_index(drop=True)
    df["raw_string"] = df["Наименование проекта/работы"]
    df["start_date"] = pd.to_datetime(parse_date(df["Начало"].astype(str)), errors='coerce')
    df["end_date"] =  pd.to_datetime(parse_date(df["Окончание"].astype(str)), errors='coerce')
    df["amount_money"] = pd.to_numeric(df["Общая стоимость"])

    return df[["raw_string", "start_date", "end_date", "amount_money"]]

def process_kr_old(df, name):
    """
    Обрабатывает kr файлы за 2017, 2018
    """
    df = df.drop(df.columns[-1], axis = 1)
    columns = df.columns = [
        "№ п/п",
        "Наименование раздела, наименование территориального учреждения, другого подразделения Банка",
        "Сметная стоимость капитального ремонта, в т.ч. ПИР",
        "Освоено",
        "Всего (утверждено)",
        "В том числе по статьям (утверждено)",
        "По объектам целевого резерва",
        "Освоено",
        "Ожидаемое освоение за год",
        "Дата завершения работ (месяц, год)",
    ]
    columns_with_amount_of_money = [
        "Сметная стоимость капитального ремонта, в т.ч. ПИР",
        "Освоено",
        "Всего (утверждено)",
        "В том числе по статьям (утверждено)",
        "По объектам целевого резерва",
        "Освоено",
        "Ожидаемое освоение за год",
    ]
    for column in columns_with_amount_of_money:
        df[column] *= 1000
    df = df[df[columns[0]].isna() == False]
    df = df.reset_index(drop=True)
    df = df.drop([i for i in range(9)])
    df = df.reset_index(drop=True)
    df["raw_string"] = df["Наименование раздела, наименование территориального учреждения, другого подразделения Банка"]
    df["start_date"] = pd.to_datetime(extract_year(name))
    df["end_date"] =  pd.to_datetime(parse_date(df["Дата завершения работ (месяц, год)"].astype(str)), errors='coerce')
    df["amount_money"] = pd.to_numeric(df["Всего (утверждено)"])

    return df[["raw_string", "start_date", "end_date", "amount_money"]]

def process_kr_2019(df, name):
    """
    Обрабатывает kr файлы за 2019
    """
    df = df.drop(df.columns[-1], axis = 1)
    columns = df.columns = [
        "№ п/п",
        "Наименование раздела, наименование территориального учреждения, другого подразделения Банка России, наименование и местонахождение объекта",
        "Сметная стоимость капитального ремонта, в т.ч. ПИР",
        "Освоено",
        "Всего(утверждено)",
        "по объектам, обеспеченным документацией",
        "по объектам целевого резерва",
        "Освоено за отчетный период",
        "Ожидаемое освоение за год",
        "Дата завершения работ (месяц, год)",
        ]
    columns_with_amount_of_money = [
        "Сметная стоимость капитального ремонта, в т.ч. ПИР",
        "Освоено",
        "Всего(утверждено)",
        "по объектам, обеспеченным документацией",
        "по объектам целевого резерва",
        "Освоено за отчетный период",
        "Ожидаемое освоение за год",
    ]
    for column in columns_with_amount_of_money:
        df[column] *= 1000
    df = df[df[columns[0]].isna() == False]
    df = df.reset_index(drop=True)
    df = df.drop([i for i in range(9)])
    df = df.reset_index(drop=True)
    df["raw_string"] = df["Наименование раздела, наименование территориального учреждения, другого подразделения Банка России, наименование и местонахождение объекта"]
    df["start_date"] = pd.to_datetime(datetime.date(year=extract_year(name), month=1, day=1))
    df["end_date"] = pd.to_datetime(parse_date(df["Дата завершения работ (месяц, год)"].astype(str)), errors="coerce")
    df["amount_money"] = pd.to_numeric(df["Всего(утверждено)"])

    return df[["raw_string", "start_date", "end_date", "amount_money"]]

def process_ks(df):
    """
    Обрабатывает ks файлы за 2020, 2021, 2022, 2023
    """
    df = df.drop(df.columns[-1], axis = 1)
    columns = df.columns = [
        "Номер проекта/ работы",
        "Наименование проекта/работы",
        "Наименование подразделения Банка России",
        "Общая стоимость",
        "Начало",
        "Окончание",
        "Всего (утверждено)",
        "В том числе по статьям 1 (утверждено)",
        "В том числе по статьям 2 (утверждено)",
        "Всего (на отчетную дату)",
        "В том числе по статьям 1 (на отчетную дату)",
        "В том числе по статьям 2 (на отчетную дату)",
        "% от годового объема затрат (расходов) по актуальной версии плана (на отчетную дату)",
        "Всего (ожидаемое исполнение)",
        "В том числе по статьям 1 (ожидаемое исполнение)",
        "В том числе по статьям 2 (ожидаемое исполнение)",
        "% от годового объема затрат (расходов) по актуальной версии плана (ожидаемое исполнение)",
        "Код состояния проекта/работы",
        "Документ",
        "Дата",
        ]
    columns_with_amount_of_money = [
       "Всего (утверждено)",
        "В том числе по статьям 1 (утверждено)",
        "В том числе по статьям 2 (утверждено)",
        "Всего (на отчетную дату)",
        "В том числе по статьям 1 (на отчетную дату)",
        "В том числе по статьям 2 (на отчетную дату)",
        "% от годового объема затрат (расходов) по актуальной версии плана (на отчетную дату)",
        "Всего (ожидаемое исполнение)",
        "В том числе по статьям 1 (ожидаемое исполнение)",
        "В том числе по статьям 2 (ожидаемое исполнение)",
        "% от годового объема затрат (расходов) по актуальной версии плана (ожидаемое исполнение)",
    ]
    for column in columns_with_amount_of_money:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    df = df[df[columns[0]].isna() == False]
    df = df.reset_index(drop=True)
    df = df.drop([0, 1, 2, 3])
    df = df.reset_index(drop=True)
    df["raw_string"] = df["Наименование проекта/работы"]
    df["start_date"] = pd.to_datetime(parse_date(df["Начало"].astype(str)))
    df["end_date"] =  pd.to_datetime(parse_date(df["Окончание"].astype(str)))
    df["amount_money"] = pd.to_numeric(df["Общая стоимость"])
    return df[["raw_string", "start_date", "end_date", "amount_money"]]

def process_ks_old(df):
    """
    Обрабатывает kr файлы за 2017, 2018, 2019
    """
    df = df.drop(df.columns[-1], axis = 1)
    columns = df.columns = [
        "Номер проекта/ работы",
        "Наименование проекта/работы",
        "Наименование подразделения Банка России",
        "Начало",
        "Окончание",
        "Общая стоимость",
        "Всего (освоено)",
        "В том числе по статьям незавершеннное строительство",
        "Всего (план)",
        "В том числе по объектам, обеспеченным документацией",
        "В том числе по объектам целевого резерва",
        "Всего (на отчетную дату)",
        "% от годового объема затрат (расходов) по актуальной версии плана (на отчетную дату)",
        "Всего (ожидаемое исполнение)",
        "% от годового объема затрат (расходов) по актуальной версии плана (ожидаемое исполнение)",
        "Объем фондов",
        "Срок ввода фондов (месяц, год)",
    ]
    columns_with_amount_of_money = [
        "Общая стоимость",
        "Всего (освоено)",
        "В том числе по статьям незавершеннное строительство",
        "Всего (план)",
        "В том числе по объектам, обеспеченным документацией",
        "В том числе по объектам целевого резерва",
        "Всего (на отчетную дату)",
        "Всего (ожидаемое исполнение)",
        "Объем фондов",
    ]
    df = df[df[columns[0]].isna() == False]
    df = df.reset_index(drop=True)
    df = df.drop([0, 1, 2, 3])
    df = df.reset_index(drop=True)
    for column in columns_with_amount_of_money:
        df[column] = pd.to_numeric(df[column], errors='coerce')
        df[column] *= 1000
    df["raw_string"] = df["Наименование проекта/работы"]
    df["start_date"] = pd.to_datetime(df["Начало"], format='mixed', errors='coerce')
    df["end_date"] = pd.to_datetime(df["Окончание"], format='mixed', errors='coerce')
    df["amount_money"] = pd.to_numeric(df["Общая стоимость"])

    return df[["raw_string", "start_date", "end_date", "amount_money"]]

def process_tr(df, name):
    """
    Обрабатывает tr файлы за 2021, 2023
    """
    df.columns = [
        "Номер п/п",
        "Местонахождение (адрес) объекта",
        "Код строки",
        "Идентификационный номер закупки по ГПЗ",
        "Сумма сметных ассигнований по Справке №1",
        "Подрядная организация",
        "№ договора, дата кем подписан",
        "Сумма договора",
        "Срок действия договора (месяц начала и месяц окончания работ MM.ГГ/MM.ГГ)",
        "Выбор подрядчика",
        "Согласование договора",
        "Заключение договора",
        "Примечания",
        "хз",
    ]
    columns_with_amount_of_money = [
        "Сумма сметных ассигнований по Справке №1",
        "Сумма договора",
    ]
    df = df.reset_index(drop=True)
    df = df.drop([i for i in range(13)])
    df = df.reset_index(drop=True)
    for column in columns_with_amount_of_money:
        df[column] *= 1000

    df["raw_string"] = df["Местонахождение (адрес) объекта"]
    df["start_date"] = pd.to_datetime(datetime.date(year=extract_year(name), month=1, day=1))
    df["end_date"] = pd.to_datetime(datetime.date(year=extract_year(name), month=12, day=31))
    df["amount_money"] = pd.to_numeric(df["Сумма сметных ассигнований по Справке №1"])

    return df[["raw_string", "start_date", "end_date", "amount_money"]]

def process_tr_old(df, name):
    """
    Обрабатывает kr файлы за 2019, 2020, 2022
    """
    df.columns = [
        "Номер п/п",
        "Наименование статей и подстатей и местонахождение объектов",
        "Код строки",
        "Утверждено согласно справке по форме 1",
        "Запланировано по объектам согласно справке по форме 1",
        "Запланировано по объектам c учетом корректировки справки по форме 1",
        "Доведено на отчетную дату",
        "Освоено с начала года",
        "Подлежит освоению",
        "Код ТУ",
        ]
    columns_with_amount_of_money = [
        "Утверждено согласно справке по форме 1",
        "Запланировано по объектам согласно справке по форме 1",
        "Запланировано по объектам c учетом корректировки справки по форме 1",
        "Доведено на отчетную дату",
        "Освоено с начала года",
        "Подлежит освоению",
    ]
    df = df.reset_index(drop=True)
    df = df.drop([i for i in range(13)])
    df = df.reset_index(drop=True)
    for column in columns_with_amount_of_money:
        df[column] *= 1000
    df["raw_string"] = df["Наименование статей и подстатей и местонахождение объектов"]
    df["start_date"] = pd.to_datetime(datetime.date(year=extract_year(name), month=1, day=1))
    df["end_date"] = pd.to_datetime(datetime.date(year=extract_year(name), month=12, day=31))
    df["amount_money"] = pd.to_numeric(df[ "Запланировано по объектам согласно справке по форме 1"])

    return df[["raw_string", "start_date", "end_date", "amount_money"]]

In [17]:
def get_kr_joined(path):
    """
    Create final kr dataframe
    with columns 
    'raw_string', 'start_date', 'end_date', 'amount_of_money'
    """
    dfs_kr = dict()
    dfs_kr_new = read_files(Path(path), f"{path}/KR.*202[0|1|2|3].*")
    dfs_kr.update({path: process_kr(df) for path, df in dfs_kr_new.items()})

    dfs_kr_old = read_files(Path(path), f"{path}/KR.*201[7|8].x.*")
    dfs_kr.update({path: process_kr_old(df, path) for path, df in dfs_kr_old.items()})

    dfs_kr_2019 = read_files(Path(path), f"{path}/KR.*2019.*")
    dfs_kr.update({path: process_kr_2019(df, path) for path, df in dfs_kr_2019.items()})

    kr_joined = pd.concat(dfs_kr, axis=0, join="outer", ignore_index=True)
    kr_joined =  kr_joined[kr_joined["amount_money"] > 0]
    kr_joined = kr_joined.dropna(subset=["raw_string"])
    kr_joined = kr_joined.drop_duplicates().reset_index(drop=True)
    return kr_joined

def get_ks_joined(path):
    """
    Create final ks dataframe
    with columns 
    'raw_string', 'start_date', 'end_date', 'amount_of_money'
    """
    dfs_ks = dict()

    dfs_ks_new = read_files(Path(path), f"{path}/KS.*202[0|1|2|3].*")
    dfs_ks.update({name: process_ks(df) for name, df in dfs_ks_new.items()})

    dfs_ks_old = read_files(Path(path), f"{path}/KC_TR_KR/KS.*20[1].*")
    dfs_ks.update({name: process_ks_old(df) for name, df in dfs_ks_old.items()})

    ks_joined = pd.concat(dfs_ks, axis=0, join="outer", ignore_index=True)
    ks_joined =  ks_joined[ks_joined["amount_money"] > 0.0]
    ks_joined = ks_joined.dropna(subset=["raw_string"])
    ks_joined = ks_joined.drop_duplicates().reset_index(drop=True)
    return ks_joined

def get_tr_joined(path):
    """
    Create final tr dataframe
    with columns 
    'raw_string', 'start_date', 'end_date', 'amount_of_money'
    """
    dfs_tr = dict()

    dfs_tr_2023 = read_files(Path(path), f"{path}/TR.*202[1|3].xls.*")
    dfs_tr.update({name: process_tr(df, name) for name, df in dfs_tr_2023.items()})

    dfs_tr_new = read_files(Path(path), f"{path}/TR.*202[0|2].xls.*")
    dfs_tr.update({name: process_tr_old(df, name) for name, df in dfs_tr_new.items()})

    dfs_tr_old = read_files(Path(path), f"{path}/TR.*2019.xls.*")
    dfs_tr.update({name: process_tr_old(df, name) for name, df in dfs_tr_old.items()})

    tr_joined = pd.concat(dfs_tr, axis=0, join="outer", ignore_index=True)
    tr_joined =  tr_joined[tr_joined["amount_money"] > 0.0]
    tr_joined = tr_joined.dropna(subset=["raw_string"])
    tr_joined = tr_joined.drop_duplicates().reset_index(drop=True)
    
    return tr_joined

In [18]:
path = "train_data/KC_TR_KR_clean"
ks_joined = get_ks_joined(path)
tr_joined = get_tr_joined(path)
kr_joined = get_kr_joined(path)

Unparsed files: 
Errors: 
Unparsed files: 
Errors: 
Unparsed files: 
Errors: 
Unparsed files: 
Errors: 
Unparsed files: 
Errors: 
Unparsed files: 
Errors: 
Unparsed files: 
Errors: 
Unparsed files: 
Errors: 


In [19]:
add_string_col_to_df(ks_joined, raw_string_column="raw_string", result_column="clean_address")
add_string_col_to_df(tr_joined)
add_string_col_to_df(kr_joined)

ks_joined = aggregate_by_month(ks_joined)
kr_joined = aggregate_by_month(kr_joined)
tr_joined = aggregate_by_month(tr_joined)


In [20]:
ks_joined

,raw_string,amount_money,clean_address,month
0,Реконструкция отдельных помещений информатизац...,1391751.035714,"г. Владивосток, Океанский проспект, д. 34",2019-11-01
1,Реконструкция отдельных помещений информатизац...,1391751.035714,"г. Владивосток, Океанский проспект, д. 34",2019-12-01
2,Реконструкция отдельных помещений информатизац...,1391751.035714,"г. Владивосток, Океанский проспект, д. 34",2020-01-01
3,Реконструкция отдельных помещений информатизац...,1391751.035714,"г. Владивосток, Океанский проспект, д. 34",2020-02-01
4,Реконструкция отдельных помещений информатизац...,1391751.035714,"г. Владивосток, Океанский проспект, д. 34",2020-03-01
...,...,...,...,...
1142,Создание системы молниезащиты на крыше здания ...,63086.0,"г. Анадырь, ул. Дежнева, 7",2020-03-01
1143,Создание системы молниезащиты на крыше здания ...,63086.0,"г. Анадырь, ул. Дежнева, 7",2020-04-01
1144,Создание системы молниезащиты на крыше здания ...,63086.0,"г. Анадырь, ул. Дежнева, 7",2020-05-01
1145,Создание системы молниезащиты на крыше здания ...,63086.0,"г. Анадырь, ул. Дежнева, 7",2020-06-01


In [21]:
kr_joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   raw_string     1228 non-null   object
 1   amount_money   1228 non-null   object
 2   clean_address  1228 non-null   object
 3   month          1228 non-null   object
dtypes: object(4)
memory usage: 38.5+ KB


In [22]:
kr_joined

,raw_string,amount_money,clean_address,month
0,Выборочный капитальный ремонт здания Централь...,171415.173913,"г. Владивосток, ул. Океанский проспект, д.34",2020-04-01
1,Выборочный капитальный ремонт здания Централь...,171415.173913,"г. Владивосток, ул. Океанский проспект, д.34",2020-05-01
2,Выборочный капитальный ремонт здания Централь...,171415.173913,"г. Владивосток, ул. Океанский проспект, д.34",2020-06-01
3,Выборочный капитальный ремонт здания Централь...,171415.173913,"г. Владивосток, ул. Океанский проспект, д.34",2020-07-01
4,Выборочный капитальный ремонт здания Централь...,171415.173913,"г. Владивосток, ул. Океанский проспект, д.34",2020-08-01
...,...,...,...,...
1223,Капитальный ремонт спортивной площадки с ограж...,854501.818182,"Приморский край, Кировский район, к.п. Горные...",2018-07-01
1224,Капитальный ремонт спортивной площадки с ограж...,854501.818182,"Приморский край, Кировский район, к.п. Горные...",2018-08-01
1225,Капитальный ремонт спортивной площадки с ограж...,854501.818182,"Приморский край, Кировский район, к.п. Горные...",2018-09-01
1226,Капитальный ремонт спортивной площадки с ограж...,854501.818182,"Приморский край, Кировский район, к.п. Горные...",2018-10-01


In [23]:
tr_joined

,raw_string,amount_money,clean_address,month
0,"Нежилые помещения г. Владивосток, ул. Светланс...",23635.454545,"г. Владивосток, ул. Светланская, 73а (51-01-01...",2020-01-01
1,"Нежилые помещения г. Владивосток, ул. Светланс...",23635.454545,"г. Владивосток, ул. Светланская, 73а (51-01-01...",2020-02-01
2,"Нежилые помещения г. Владивосток, ул. Светланс...",23635.454545,"г. Владивосток, ул. Светланская, 73а (51-01-01...",2020-03-01
3,"Нежилые помещения г. Владивосток, ул. Светланс...",23635.454545,"г. Владивосток, ул. Светланская, 73а (51-01-01...",2020-04-01
4,"Нежилые помещения г. Владивосток, ул. Светланс...",23635.454545,"г. Владивосток, ул. Светланская, 73а (51-01-01...",2020-05-01
...,...,...,...,...
2800,1. Часть-здания - общежитие по адресу: ул. 50 ...,14454.545455,"г. Якутск, Республика Саха (Якутия). ...",2018-07-01
2801,1. Часть-здания - общежитие по адресу: ул. 50 ...,14454.545455,"г. Якутск, Республика Саха (Якутия). ...",2018-08-01
2802,1. Часть-здания - общежитие по адресу: ул. 50 ...,14454.545455,"г. Якутск, Республика Саха (Якутия). ...",2018-09-01
2803,1. Часть-здания - общежитие по адресу: ул. 50 ...,14454.545455,"г. Якутск, Республика Саха (Якутия). ...",2018-10-01


In [24]:
save_dfs({"tr_joined": tr_joined}, path="parsed")
save_dfs({"ks_joined": ks_joined}, path="parsed")
save_dfs({"kr_joined": kr_joined}, path="parsed")

In [25]:
shutil.make_archive("parsed", 'zip', "parsed")

'/home/step/backend/parsed.zip'

In [26]:

def get_unique_keys(expenses_addresses, tr_joined, ks_joined, kr_joined, df_list_on):

    return set(itertools.chain(
        tr_joined["clean_address"],
        ks_joined["clean_address"],
        kr_joined["clean_address"],
        df_list_on["clean_address"],
        expenses_addresses["expenses_address"],
    ))


In [28]:
expenses_addresses = pd.read_excel("expenses_addresses.xlsx")
df_list_on = pd.read_feather("data_lists_on.feather")

all_raw_strings = get_unique_keys(expenses_addresses, tr_joined, ks_joined, kr_joined, df_list_on)
len(all_raw_strings)

1775

In [30]:
len(expenses_addresses)

386

In [31]:
def request_to_geocoder(string, api_key="8c1a744e-abae-4b1f-a292-076e70d90d92"):
    lat_1, lon_1 = 41.755060, 100.567964
    lat_2, lon_2 = 78.300151, 190.346714
    query = f"https://geocode-maps.yandex.ru/1.x?apikey={api_key}&geocode={string}&format=json&bbox={lon_1},{lat_1}~{lon_2},{lat_2}"
    res = requests.get(query)
    return json.loads(res.content)


def extract_lat_lon(json_response):
    try:
        lon, lat = map(float, json_response["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["Point"]["pos"].split())
        return lat, lon
    except Exception as exc:
        print(exc)
        return None, None


def extract_address(json_response):
    try:
        address = json_response["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["metaDataProperty"]["GeocoderMetaData"]["AddressDetails"]["Country"]["AddressLine"]
        return address
    except Exception as exc:
        print(exc)
        return None
    
def check_conditions(json_response):
    try:
        code = json_response["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["metaDataProperty"]["GeocoderMetaData"]["AddressDetails"]["Country"]["CountryNameCode"]
        return code == 'RU'
    except Exception as exc:
        print(exc)
        return None

def save_dict(path, dictionary):
    with open(path, 'wb') as f:
        pickle.dump(dictionary, f)

def read_dict(path):
    with open(path, 'rb') as f:
        return pickle.loads(f.read())

def get_appliable_function_and_geocoder_results(geocoder_results_path):
    skipped = set()
    geocoder_results = read_dict(geocoder_results_path)
    def get_geocoder_features(row):
        json_response = geocoder_results[row["raw_string"]]
        if check_conditions(json_response):
            row["geocoder_address"] = extract_address(json_response)
            lat, lon = extract_lat_lon(json_response)
            row["geocoder_lat"] = lat
            row["geocoder_lon"] = lon

            return row
        row["geocoder_lat"] = None
        row["geocoder_lon"] = None
        row["geocoder_address"] = None
        return row

    return get_geocoder_features, geocoder_results, skipped


def get_geocoder_results(all_raw_strings, api_key):
    geocoder_results = dict()
    for raw_string in all_raw_strings:
        geocoder_results[raw_string] = request_to_geocoder(raw_string, api_key)
    return geocoder_results

In [32]:
geocoder_results = get_geocoder_results(all_raw_strings, api_key="dccd8a7c-0ab2-466e-946e-90ae3d95baec")
save_dict("geocoder_results_new.pkl", geocoder_results)

In [33]:
get_geocoder_features, geocoder_results, skipped = get_appliable_function_and_geocoder_results("geocoder_results_new.pkl")
geocoder_df = pd.DataFrame()
geocoder_df["raw_string"] = geocoder_results.keys()
geocoder_df = geocoder_df.apply(get_geocoder_features, axis=1)


'response'
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [34]:
geocoder_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1775 entries, 0 to 1774
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   geocoder_address  1724 non-null   object 
 1   geocoder_lat      1724 non-null   float64
 2   geocoder_lon      1724 non-null   float64
 3   raw_string        1774 non-null   object 
dtypes: float64(2), object(2)
memory usage: 55.6+ KB


In [35]:

# def natasha_process(text):
#      if not text:
#           return None
#      segmenter = Segmenter()
#      morph_vocab = MorphVocab()
#      addr_extractor = AddrExtractor(morph_vocab)

#      matches = addr_extractor(text)
#      facts = [i.fact.as_json for i in matches]
#      lst = []
#      for fact in facts:
#           lst.extend(list(fact.values())[::-1])
#      return " ".join(lst)

for elem in skipped:
     print(elem, "===========", natasha_process(elem))

In [36]:
extract_string_for_search(" Капитальный ремонт спортивной площадки с ограждением, расположенной по адресу: Приморский край, Кировский район, к.п. Горные Ключи, ул. Цымбалюка, 1")

' Приморский край, Кировский район, к.п. Горные Ключи, ул. Цымбалюка, 1'

In [56]:
geocoder_df["geocoder_address"].nunique()

334

In [58]:
geocoder_df

,geocoder_address,geocoder_lat,geocoder_lon,raw_string
0,None,NaN,NaN,
1,"Россия, Еврейская автономная область, Биробидж...",48.789445,132.928222,"""Город Биробиджа"", город Биробиджан, проспек..."
2,None,NaN,NaN,"6. Наружное освещение сан. зоны санатория ""им...."
3,"Россия, Приморский край, Уссурийск, улица Некр...",43.805100,131.950262,"г. Уссурийск, ул. Некрасова, д. 102"
4,"Россия, Хабаровский край, Советская Гавань, Со...",48.965926,140.281571,"г. Советская Гавань, ул. Советская, 26"
...,...,...,...,...
1770,"Россия, Еврейская автономная область, Биробидж...",48.789445,132.928222,"г Биробиджан, пр-кт 60-летия СССР, д 5"
1771,"Россия, Камчатский край, Петропавловск-Камчатс...",53.068444,158.628735,"г. Петропавловск-Камчатский, ул. Ломоносова,..."
1772,"Россия, Сахалинская область, Южно-Сахалинск, К...",46.958281,142.732004,"г. Южно-Сахалинск, Коммунистический проспект, ..."
1773,"Россия, Чукотский автономный округ, городской ...",64.725052,177.525974,ул колхозная 30б


In [64]:
def get_weather_df(geocoder_df):
    new_dfs = []
    start = datetime.datetime(2016, 1, 1)
    end = datetime.datetime(2023, 10, 31)
    processed = set()
    for _, row in geocoder_df.iterrows():
        if row["geocoder_address"] in processed:
            continue
        if row["geocoder_lat"] is not None and row["geocoder_lon"] is not None:
            processed.add(row["geocoder_address"])
            point = Point(
                lat=row["geocoder_lat"],
                lon=row["geocoder_lat"],
            )
            df = Monthly(point, start, end)
            df = df.fetch()
            df["geocoder_address"] = row["geocoder_address"]
            new_dfs.append(df)

    return pd.concat(new_dfs, axis=0)

In [65]:
weather_df = get_weather_df(geocoder_df)

In [66]:
weather_df

,tavg,tmin,tmax,prcp,wspd,pres,tsun,geocoder_address
2016-01-01,-2.2,-5.7,1.7,48.1,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр..."
2016-02-01,3.4,NaN,NaN,NaN,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр..."
2016-03-01,5.3,NaN,NaN,NaN,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр..."
2016-04-01,12.7,NaN,NaN,NaN,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр..."
2016-05-01,15.8,NaN,NaN,NaN,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр..."
...,...,...,...,...,...,...,...,...
2023-06-01,20.7,12.9,27.1,62.9,15.3,1011.1,NaN,"Россия, Хабаровский край, Верхнебуреинский рай..."
2023-07-01,23.9,16.9,30.0,112.3,12.7,1007.4,NaN,"Россия, Хабаровский край, Верхнебуреинский рай..."
2023-08-01,22.8,14.6,29.9,11.1,12.8,1012.2,NaN,"Россия, Хабаровский край, Верхнебуреинский рай..."
2023-09-01,15.4,7.7,22.9,13.2,9.0,1019.7,NaN,"Россия, Хабаровский край, Верхнебуреинский рай..."


In [67]:
weather_df["month"] = weather_df.index
weather_df["key_merge"] = weather_df["geocoder_address"] + "_" + weather_df["month"].astype(str)
weather_df["key_merge"].nunique()


5423

In [68]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5423 entries, 2016-01-01 to 2023-10-01
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   tavg              4829 non-null   float64       
 1   tmin              4011 non-null   float64       
 2   tmax              4248 non-null   float64       
 3   prcp              3772 non-null   float64       
 4   wspd              2189 non-null   float64       
 5   pres              2518 non-null   float64       
 6   tsun              980 non-null    float64       
 7   geocoder_address  5423 non-null   object        
 8   month             5423 non-null   datetime64[ns]
 9   key_merge         5423 non-null   object        
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 466.0+ KB


In [69]:
weather_df["month"] = weather_df.index
weather_df = weather_df.reset_index(drop=True)
weather_df

,tavg,tmin,tmax,prcp,wspd,pres,tsun,geocoder_address,month,key_merge
0,-2.2,-5.7,1.7,48.1,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр...",2016-01-01,"Россия, Приморский край, Уссурийск, улица Некр..."
1,3.4,NaN,NaN,NaN,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр...",2016-02-01,"Россия, Приморский край, Уссурийск, улица Некр..."
2,5.3,NaN,NaN,NaN,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр...",2016-03-01,"Россия, Приморский край, Уссурийск, улица Некр..."
3,12.7,NaN,NaN,NaN,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр...",2016-04-01,"Россия, Приморский край, Уссурийск, улица Некр..."
4,15.8,NaN,NaN,NaN,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр...",2016-05-01,"Россия, Приморский край, Уссурийск, улица Некр..."
...,...,...,...,...,...,...,...,...,...,...
5418,20.7,12.9,27.1,62.9,15.3,1011.1,NaN,"Россия, Хабаровский край, Верхнебуреинский рай...",2023-06-01,"Россия, Хабаровский край, Верхнебуреинский рай..."
5419,23.9,16.9,30.0,112.3,12.7,1007.4,NaN,"Россия, Хабаровский край, Верхнебуреинский рай...",2023-07-01,"Россия, Хабаровский край, Верхнебуреинский рай..."
5420,22.8,14.6,29.9,11.1,12.8,1012.2,NaN,"Россия, Хабаровский край, Верхнебуреинский рай...",2023-08-01,"Россия, Хабаровский край, Верхнебуреинский рай..."
5421,15.4,7.7,22.9,13.2,9.0,1019.7,NaN,"Россия, Хабаровский край, Верхнебуреинский рай...",2023-09-01,"Россия, Хабаровский край, Верхнебуреинский рай..."


In [70]:
weather_df.to_feather("weather_results_final.frt")

In [41]:
geocoder_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1775 entries, 0 to 1774
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   geocoder_address  1724 non-null   object 
 1   geocoder_lat      1724 non-null   float64
 2   geocoder_lon      1724 non-null   float64
 3   raw_string        1774 non-null   object 
dtypes: float64(2), object(2)
memory usage: 55.6+ KB


In [42]:
geocoder_df.to_feather("geocoder_results_final.frt")

In [43]:
ks_joined

,raw_string,amount_money,clean_address,month
0,Реконструкция отдельных помещений информатизац...,1391751.035714,"г. Владивосток, Океанский проспект, д. 34",2019-11-01
1,Реконструкция отдельных помещений информатизац...,1391751.035714,"г. Владивосток, Океанский проспект, д. 34",2019-12-01
2,Реконструкция отдельных помещений информатизац...,1391751.035714,"г. Владивосток, Океанский проспект, д. 34",2020-01-01
3,Реконструкция отдельных помещений информатизац...,1391751.035714,"г. Владивосток, Океанский проспект, д. 34",2020-02-01
4,Реконструкция отдельных помещений информатизац...,1391751.035714,"г. Владивосток, Океанский проспект, д. 34",2020-03-01
...,...,...,...,...
1142,Создание системы молниезащиты на крыше здания ...,63086.0,"г. Анадырь, ул. Дежнева, 7",2020-03-01
1143,Создание системы молниезащиты на крыше здания ...,63086.0,"г. Анадырь, ул. Дежнева, 7",2020-04-01
1144,Создание системы молниезащиты на крыше здания ...,63086.0,"г. Анадырь, ул. Дежнева, 7",2020-05-01
1145,Создание системы молниезащиты на крыше здания ...,63086.0,"г. Анадырь, ул. Дежнева, 7",2020-06-01


In [44]:
res = request_to_geocoder("улица Ленина", api_key="dccd8a7c-0ab2-466e-946e-90ae3d95baec")
res

{'response': {'GeoObjectCollection': {'metaDataProperty': {'GeocoderResponseMetaData': {'boundedBy': {'Envelope': {'lowerCorner': '100.567968 41.740224',
       'upperCorner': '190.346712 78.304168'}},
     'request': 'улица Ленина',
     'results': '10',
     'found': '10'}},
   'featureMember': [{'GeoObject': {'metaDataProperty': {'GeocoderMetaData': {'precision': 'street',
        'text': 'Россия, Амурская область, Благовещенск, улица Ленина',
        'kind': 'street',
        'Address': {'country_code': 'RU',
         'formatted': 'Россия, Амурская область, Благовещенск, улица Ленина',
         'Components': [{'kind': 'country', 'name': 'Россия'},
          {'kind': 'province', 'name': 'Дальневосточный федеральный округ'},
          {'kind': 'province', 'name': 'Амурская область'},
          {'kind': 'area', 'name': 'городской округ Благовещенск'},
          {'kind': 'locality', 'name': 'Благовещенск'},
          {'kind': 'street', 'name': 'улица Ленина'}]},
        'AddressDetails

In [45]:
df = pd.read_feather("geocoder_results.frt")
df["geocoder_address"].nunique()

348

In [46]:
res = request_to_geocoder("Москва", api_key="5091a209-95f3-4dd4-a66d-b271fbb83f47")
res

{'response': {'GeoObjectCollection': {'metaDataProperty': {'GeocoderResponseMetaData': {'boundedBy': {'Envelope': {'lowerCorner': '37.038186 55.312148',
       'upperCorner': '38.2026 56.190802'}},
     'request': 'Москва',
     'results': '10',
     'found': '1'}},
   'featureMember': [{'GeoObject': {'metaDataProperty': {'GeocoderMetaData': {'precision': 'other',
        'text': 'Россия, Москва',
        'kind': 'province',
        'Address': {'country_code': 'RU',
         'formatted': 'Россия, Москва',
         'Components': [{'kind': 'country', 'name': 'Россия'},
          {'kind': 'province', 'name': 'Центральный федеральный округ'},
          {'kind': 'province', 'name': 'Москва'}]},
        'AddressDetails': {'Country': {'AddressLine': 'Россия, Москва',
          'CountryNameCode': 'RU',
          'CountryName': 'Россия',
          'AdministrativeArea': {'AdministrativeAreaName': 'Москва'}}}}},
      'name': 'Москва',
      'description': 'Россия',
      'boundedBy': {'Envelope'

In [47]:
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2022, 1, 31)

# Create Point for Vancouver, BC
khabarovsk = Point(lat=48.478, lon=135.080047)

# Get daily data for 2018
data = Monthly(khabarovsk, start, end)
data = data.fetch()

# Plot line chart including average, minimum and maximum temperature
data

,tavg,tmin,tmax,prcp,wspd,pres,tsun
time,,,,,,,
2018-01-01,-19.8,-21.8,-16.0,18.5,NaN,NaN,NaN
2018-02-01,-17.1,-21.4,-12.5,2.3,14.9,NaN,NaN
2018-03-01,-5.7,-10.3,-1.0,21.9,17.7,NaN,NaN
2018-04-01,6.4,1.2,12.6,30.0,NaN,NaN,NaN
2018-05-01,14.0,7.3,20.4,23.8,NaN,NaN,NaN
2018-06-01,17.0,11.9,22.2,207.3,NaN,NaN,NaN
2018-07-01,21.8,17.7,26.4,161.4,NaN,1007.3,NaN
2018-08-01,19.8,15.3,24.6,58.5,10.7,1008.8,NaN
2018-09-01,13.4,8.9,19.0,131.2,13.4,1010.6,NaN


In [48]:
a

NameError: name 'a' is not defined

In [ ]:
df

,raw_string,geocoder_address,geocoder_lat,geocoder_lon
0,,None,NaN,NaN
1,"г. Анадырь, ул. Рультытегина, д.17 кв.6","Россия, Чукотский автономный округ, Анадырь, у...",64.733933,177.512580
2,"г.о. Владивостокский, г. Владивосток, ул. С...","Россия, Приморский край, Владивосток, Семёновс...",43.118675,131.889285
3,"г. Петропавловск-Камчатский, ул. Ломоносова,...","Россия, Камчатский край, Петропавловск-Камчатс...",53.068444,158.628735
4,"р-н. Олюторский, с. Тиличики, ул. Школьная,...","Россия, Камчатский край, Олюторский район, сел...",60.427389,166.052044
...,...,...,...,...
1780,"город Хабаровск, р-н Центральный, ул. Мурав...","Россия, Хабаровск, улица Муравьёва-Амурского, 42",48.477751,135.067820
1781,"г. Хабаровск, участок находится примерно в ...","Россия, Хабаровск, Садоводческое некоммерческо...",48.458252,135.195687
1782,"город Хабаровск, ул. Калинина, д. 156, Лит...","Россия, Хабаровск, улица Калинина, 156",48.484595,135.049863
1783,"г. Тында, Профсоюзная ул., д.12","Россия, Амурская область, Тында, Профсоюзная у...",55.148632,124.738228


In [ ]:
for elem in kr_joined["clean_address"]:
    if elem not in set(geocoder_df["raw_string"]):
        print(elem)

In [ ]:
for elem in tr_joined["clean_address"]:
    if elem not in set(geocoder_df["raw_string"]):
        print(elem)

In [ ]:
for elem in ks_joined["clean_address"]:
    if elem not in set(geocoder_df["raw_string"]):
        print(elem)

In [71]:
data = pd.read_feather("server/data_with_target.frt")

In [85]:
data[["Техническое состояние"]].info()

<class 'pandas.core.frame.DataFrame'>
Index: 6264 entries, 0 to 6263
Data columns (total 1 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Техническое состояние  4665 non-null   object
dtypes: object(1)
memory usage: 97.9+ KB


In [88]:
data.columns

Index(['month', 'TARGET', 'geocoder_address', 'geocoder_lat', 'geocoder_lon',
       'area', 'key_merge', 'amount_money_kr', 'amount_money_ks',
       'amount_money_tr', 'tavg', 'tmin', 'tmax', 'prcp', 'wspd', 'pres',
       'tsun', 'Unnamed: 0', 'Наименование, адрес, реестровый номер объекта',
       'Год постройки', 'Дата принятия на баланс Банка России',
       'Площадь объекта недвижимости \nкв. м', 'Занято службами Банка России',
       'Не занято (не исполь\nзуется)', 'Сдается Банком России в аренду',
       'Аренду\nется Банком России',
       'Передано Банку России по договору безвозмездного пользо-вания',
       'Передано в субаренду', 'Балансовая стоимость, руб.',
       'в том числе переоценка, руб.', 'Сумма начисленной амортизации, руб.',
       'Остаточная стоимость, руб.', 'Техническое состояние',
       'ГЕОГРАФИЧЕСКОЕ НАЗВАНИЕ', 'ПУТЬ', 'ВОЗРАСТ ЗДАНИЯ', 'ГОД', 'КВАРТАЛ',
       'clean_address', 'code', 'raw_string'],
      dtype='object')

In [90]:
train = data[["month", # месяц
      "TARGET", # prediction
      "geocoder_lat", # lat
      "geocoder_lon", # lon
      "amount_money_kr", # krValue
      "amount_money_ks", # ksValue
      "amount_money_tr", # trValue
      "tavg",
      "tmin", 
      "tmax",
      "prcp", # осадки
      "Техническое состояние", # technicalConditions
      "ВОЗРАСТ ЗДАНИЯ", # buildingAge
      'Площадь объекта недвижимости \nкв. м', # buildingSquare
      "Остаточная стоимость, руб.", # residualValue
      "Балансовая стоимость, руб.", # balanceValue
]]

In [92]:
train["Техническое состояние"].unique()

array(['Удовлетворительное', 'Хорошее', None, 'Аварийное', 'Ветхое'],
      dtype=object)

In [97]:
def _compute_buildings(data):
    buildings = dict()
    processed = set()
    for _, row in data.iterrows():
        geocoder_address = row["geocoder_address"]
        if geocoder_address in processed:
            continue
        row_to_insert = dict(
            latitude=row["geocoder_lat"],
            longitude=row["geocoder_lon"],
            buildingAge=row["ВОЗРАСТ ЗДАНИЯ"],
            buildingSquare=row["Площадь объекта недвижимости \nкв. м"],
            technicalConditions=row["Техническое состояние"],
            krValue=row["amount_money_kr"],
            trValue=row["amount_money_ks"],
            ksValue=row["amount_money_ks"]
        )
        buildings[geocoder_address] = row_to_insert
    return buildings

In [98]:
_compute_buildings(data)

ValueError: Must have equal len keys and value when setting with an iterable

In [96]:
data.columns

Index(['month', 'TARGET', 'geocoder_address', 'geocoder_lat', 'geocoder_lon',
       'area', 'key_merge', 'amount_money_kr', 'amount_money_ks',
       'amount_money_tr', 'tavg', 'tmin', 'tmax', 'prcp', 'wspd', 'pres',
       'tsun', 'Unnamed: 0', 'Наименование, адрес, реестровый номер объекта',
       'Год постройки', 'Дата принятия на баланс Банка России',
       'Площадь объекта недвижимости \nкв. м', 'Занято службами Банка России',
       'Не занято (не исполь\nзуется)', 'Сдается Банком России в аренду',
       'Аренду\nется Банком России',
       'Передано Банку России по договору безвозмездного пользо-вания',
       'Передано в субаренду', 'Балансовая стоимость, руб.',
       'в том числе переоценка, руб.', 'Сумма начисленной амортизации, руб.',
       'Остаточная стоимость, руб.', 'Техническое состояние',
       'ГЕОГРАФИЧЕСКОЕ НАЗВАНИЕ', 'ПУТЬ', 'ВОЗРАСТ ЗДАНИЯ', 'ГОД', 'КВАРТАЛ',
       'clean_address', 'code', 'raw_string'],
      dtype='object')